<a href="https://colab.research.google.com/github/denisabrantesredis/denisd-redis-learning-sessions/blob/main/ClientSideCaching/ClientSideCaching.ipynb" target="_newt">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

<div style="display:flex;width=100%;">
<img src="https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120" alt="Redis" width="90"/>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
</div>

# Redis Learning Session - Client-Side Caching

<img src="https://redis.io/docs/latest/images/csc/CSCNoCache.drawio.svg" alt="CSC - No Cache"/>
<br/><br/><br/><br/>
<img src="https://redis.io/docs/latest/images/csc/CSCWithCache.drawio.svg" alt="CSC - Cache"/>

[Try a sample Java application](https://github.com/Redislabs-Solution-Architects/redis-client-side-caching-csc-jedis-demo)

In this notebook, we will explore the Client-Side Caching capabilities provided by Redis.

## Before We Start

<a href="https://redis.io/try-free/" target="_new">
<img src="https://github.com/denisabrantesredis/denisd-redis-learning-sessions/blob/main/ClientSideCaching/_assets/images/callout_rediscloud.png?raw=true" alt="Callout - Create free Redis Cloud Account"/>
</a>

<b>Create a new free Redis Cloud account: <a href="https://redis.io/try-free/" target="_new">Click Here</a>

&nbsp;
&nbsp;
<a href="https://colab.research.google.com/github/denisabrantesredis/denisd-redis-learning-sessions/blob/main/ClientSideCaching/ClientSideCachingMonitor.ipynb" target="_newt">
<img src="https://github.com/denisabrantesredis/denisd-redis-learning-sessions/blob/main/ClientSideCaching/_assets/images/callout_monitor.png?raw=true" alt="Callout - Open Monitor Notebook"/>
</a>

Open the Monitor Notebook in a new tab to keep track of the commands executed by the Redis Server.

## Installing the Pre-Reqs

In [1]:
!pip install -q redis
!pip install -q ipython-autotime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.2/354.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.7 MB/s eta 0:00:00


In [2]:
%load_ext autotime

time: 339 µs (started: 2026-01-27 13:34:47 +00:00)


## Installing Redis Locally
If you are not using Redis Cloud as a database, uncomment and run the code below to install Redis locally. Then set your connection to 127.0.0.1

In [ ]:
# %%sh
# sudo apt-get install lsb-release curl gpg
# curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
# sudo chmod 644 /usr/share/keyrings/redis-archive-keyring.gpg
# echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
# sudo apt-get update
# sudo apt-get install redis > /dev/null 2>&1
# redis-server --daemonize yes

## Copying and Unzipping Lab Files

In [3]:
import os

time: 328 µs (started: 2026-01-27 13:34:51 +00:00)


In [4]:
if not os.path.exists("./files"):
  !mkdir files
  !wget https://github.com/denisabrantesredis/denisd-redis-learning-sessions/raw/refs/heads/main/Search/_assets/files/lab_assets.zip
  !mv lab_assets.zip ./files
  !unzip ./files/lab_assets.zip -d ./files

--2026-01-27 13:34:55--  https://github.com/denisabrantesredis/denisd-redis-learning-sessions/raw/refs/heads/main/Search/_assets/files/lab_assets.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/denisabrantesredis/denisd-redis-learning-sessions/refs/heads/main/Search/_assets/files/lab_assets.zip [following]
--2026-01-27 13:34:56--  https://raw.githubusercontent.com/denisabrantesredis/denisd-redis-learning-sessions/refs/heads/main/Search/_assets/files/lab_assets.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3718910 (3.5M) [application/zip]
Saving to: ‘lab_assets.zip’

lab_assets.zip     

## Connecting to Redis

In [5]:
import redis
from redis.cache import CacheConfig, EvictionPolicy

from google.colab import userdata

time: 63.4 ms (started: 2026-01-27 13:35:01 +00:00)


#### Setup the Connection String

<img src="https://github.com/denisabrantesredis/denisd-redis-learning-sessions/blob/main/Streams/_assets/images/callout_secrets.png?raw=true" alt="Callout - Use Google Colab secrets instead"/>

In [6]:
try:
  REDIS_HOST = userdata.get('REDIS_HOST')
except:
  REDIS_HOST="127.0.0.1"

try:
  REDIS_PORT = userdata.get('REDIS_PORT')
except:
  REDIS_PORT=6379

try:
  REDIS_PASSWORD = userdata.get('REDIS_PASSWORD')
except:
  REDIS_PASSWORD=""

REDIS_URL = f"redis://default:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

time: 485 ms (started: 2026-01-27 13:35:04 +00:00)


#### Testing the Connection to Redis

<img src="https://github.com/denisabrantesredis/denisd-redis-learning-sessions/blob/main/Streams/_assets/images/callout_connection.png?raw=true" alt="Callout - Make sure connection works"/>

In [7]:
r = redis.from_url(
    REDIS_URL,
    protocol=3,
    cache_config=CacheConfig(max_size=10, eviction_policy=EvictionPolicy.LRU),
    decode_responses=True)

if r.ping():
    print("Connection successful!")
else:
    print("Connection issue!")

Connection successful!
time: 153 ms (started: 2026-01-27 13:35:06 +00:00)


<img src="https://github.com/denisabrantesredis/denisd-redis-learning-sessions/blob/main/Streams/_assets/images/callout_insight.png?raw=true" alt="Callout - Check Redis Insight"/>

Open Redis Insight and confirm that the Stream key was generated. There should be only one key, called `mystream`, which contains all the 4 entries we generated, since no consumer has processed these messages yet.

Needless to say, we shouldn't be inserting completely different messages to the same stream; we will fix that in a bit.

&nbsp;

&nbsp;

## Caching String Data

First, let's test Client-Side Caching for Strings. We'll start by setting a simple string:

In [8]:
r.set("myname", "My Name")

True

time: 21.9 ms (started: 2026-01-27 13:35:11 +00:00)


Next, read the value of the key:

In [9]:
r.get("myname")

'My Name'

time: 21.8 ms (started: 2026-01-27 13:35:24 +00:00)


Reading it for a second time should reduce the execution time, because the key will be retrieved from the local cache:

In [15]:
r.get("myname")

'My Name'

time: 2.57 ms (started: 2026-01-27 13:35:49 +00:00)


Check the MONITOR output in the other notebook; only 1 GET command should be listed there.

## Caching Streams data

Let's store the Streams dataset we used in the Search lab:

In [17]:
from datetime import datetime, timedelta
import pandas as pd
import pytz

time: 414 ms (started: 2026-01-27 13:38:44 +00:00)


Load data from CSV:

In [18]:
iot_ds = pd.read_csv('./files/iot.csv')
print(len(iot_ds))
iot_ds.head()

97546


,Unnamed: 0,id,room,date,temp,location,timestamp
0,0,__export__.temp_log_196134_bd201015,Room Admin,01-03-2024 16:02:00,29,In,1709330520
1,1,__export__.temp_log_196131_7bca51bc,Room Admin,01-03-2024 16:03:00,29,In,1709330580
2,2,__export__.temp_log_196127_522915e3,Room Admin,01-03-2024 16:04:00,41,Out,1709330640
3,3,__export__.temp_log_196128_be0919cf,Room Admin,01-03-2024 16:05:00,41,Out,1709330700
4,4,__export__.temp_log_196126_d30b72fb,Room Admin,01-03-2024 16:06:00,31,In,1709330760


time: 556 ms (started: 2026-01-27 13:38:52 +00:00)


Using a pipeline to save data to Redis: all messages will be under a single stream (key), named `stream:iot`. The pipeline will gather all commands and execute them once against Redis.

In [19]:
pipe = r.pipeline(transaction=False)
keyname = "streams:iot"
for index, row in iot_ds.iterrows():
      value = {
            'id': row['id'],
            'room': row['room'],
            'date': row['date'],
            'temp' : row['temp'],
            'location': row['location'],
            'timestamp': row['timestamp']
      }
      pipe.xadd(keyname, id=row['timestamp'], fields=value)
result = pipe.execute()
len(result)

97546

time: 12.5 s (started: 2026-01-27 13:39:53 +00:00)


<img src="https://github.com/denisabrantesredis/denisd-redis-learning-sessions/blob/main/Streams/_assets/images/callout_insight.png?raw=true" alt="Callout - Check Redis Insight"/>

Open Redis Insight and confirm that the Stream key was generated. The key should be called `streams:iot`, and it contains about 60,000 messages.

&nbsp;

&nbsp;

Let's check the date ranges available in this dataset:

In [20]:
print(iot_ds.iloc[0]['date'])
print(iot_ds.iloc[len(iot_ds)-1]['date'])

01-03-2024 16:02:00 
08-05-2024 10:47:00 
time: 1.22 ms (started: 2026-01-27 13:41:53 +00:00)


Define the date range for our Streams search:

<img src="https://github.com/denisabrantesredis/denisd-redis-learning-sessions/blob/main/Search/_assets/images/callout_streams.png?raw=true" alt="Callout - Change Time Range"/>

In [21]:
start_date = "01-04-2024 12:00"
end_date = "01-04-2024 12:30"

time: 676 µs (started: 2026-01-27 13:42:33 +00:00)


Transform these strings into timestamps:

In [22]:
datetime_format = "%d-%m-%Y %H:%M"
local_tz = pytz.timezone('America/Chicago')

local_start = datetime.strptime(start_date, datetime_format)
utc_start = local_tz.localize(local_start)
first_ts = utc_start.timestamp()

local_end = datetime.strptime(end_date, datetime_format)
utc_end = local_tz.localize(local_end)
last_ts = utc_end.timestamp()

time: 31.9 ms (started: 2026-01-27 13:42:50 +00:00)


Find all Streams messages within this time range:

In [23]:
messages = r.xrange("streams:iot", int(first_ts), int(last_ts))
for message in messages:
  print(message)

('1711990800-0', {'id': '__export__.temp_log_72242_28916895', 'room': 'Room Admin', 'date': '01-04-2024 12:00:00 ', 'temp': '42', 'location': 'Out', 'timestamp': '1711990800'})
('1711990860-0', {'id': '__export__.temp_log_51697_51123bd0', 'room': 'Room Admin', 'date': '01-04-2024 12:01:00 ', 'temp': '42', 'location': 'Out', 'timestamp': '1711990860'})
('1711990920-0', {'id': '__export__.temp_log_52228_379fb5ff', 'room': 'Room Admin', 'date': '01-04-2024 12:02:00 ', 'temp': '42', 'location': 'Out', 'timestamp': '1711990920'})
('1711990980-0', {'id': '__export__.temp_log_52164_2d69b49b', 'room': 'Room Admin', 'date': '01-04-2024 12:03:00 ', 'temp': '42', 'location': 'Out', 'timestamp': '1711990980'})
('1711991040-0', {'id': '__export__.temp_log_71376_24a1629c', 'room': 'Room Admin', 'date': '01-04-2024 12:04:00 ', 'temp': '42', 'location': 'Out', 'timestamp': '1711991040'})
('1711991100-0', {'id': '__export__.temp_log_51693_1c235ce5', 'room': 'Room Admin', 'date': '01-04-2024 12:05:00 ',

Check the MONITOR notebook for the XRANGE output; now run the command again:

In [24]:
messages = r.xrange("streams:iot", int(first_ts), int(last_ts))
for message in messages:
  print(message)

('1711990800-0', {'id': '__export__.temp_log_72242_28916895', 'room': 'Room Admin', 'date': '01-04-2024 12:00:00 ', 'temp': '42', 'location': 'Out', 'timestamp': '1711990800'})
('1711990860-0', {'id': '__export__.temp_log_51697_51123bd0', 'room': 'Room Admin', 'date': '01-04-2024 12:01:00 ', 'temp': '42', 'location': 'Out', 'timestamp': '1711990860'})
('1711990920-0', {'id': '__export__.temp_log_52228_379fb5ff', 'room': 'Room Admin', 'date': '01-04-2024 12:02:00 ', 'temp': '42', 'location': 'Out', 'timestamp': '1711990920'})
('1711990980-0', {'id': '__export__.temp_log_52164_2d69b49b', 'room': 'Room Admin', 'date': '01-04-2024 12:03:00 ', 'temp': '42', 'location': 'Out', 'timestamp': '1711990980'})
('1711991040-0', {'id': '__export__.temp_log_71376_24a1629c', 'room': 'Room Admin', 'date': '01-04-2024 12:04:00 ', 'temp': '42', 'location': 'Out', 'timestamp': '1711991040'})
('1711991100-0', {'id': '__export__.temp_log_51693_1c235ce5', 'room': 'Room Admin', 'date': '01-04-2024 12:05:00 ',

These results are served from the cache.

Keep in mind that the entire key is cached, so a new entry will trigger the invalidation of the cache, even a partial one. Let's add a new message to this stream:

In [25]:
r.xadd("streams:iot", id="*",
       fields={'id': 'temp_log_51679_4928200b', 'room': 'Room Admin', 'date': '01-02-2026 12:30:00 ', 'temp': '45', 'location': 'Out', 'timestamp': '1711992601'})

'1769521604977-0'

time: 23.1 ms (started: 2026-01-27 13:46:44 +00:00)


Next, let's get the same range again.

In [26]:
messages = r.xrange("streams:iot", int(first_ts), int(last_ts))
for message in messages:
  print(message)

('1711990800-0', {'id': '__export__.temp_log_72242_28916895', 'room': 'Room Admin', 'date': '01-04-2024 12:00:00 ', 'temp': '42', 'location': 'Out', 'timestamp': '1711990800'})
('1711990860-0', {'id': '__export__.temp_log_51697_51123bd0', 'room': 'Room Admin', 'date': '01-04-2024 12:01:00 ', 'temp': '42', 'location': 'Out', 'timestamp': '1711990860'})
('1711990920-0', {'id': '__export__.temp_log_52228_379fb5ff', 'room': 'Room Admin', 'date': '01-04-2024 12:02:00 ', 'temp': '42', 'location': 'Out', 'timestamp': '1711990920'})
('1711990980-0', {'id': '__export__.temp_log_52164_2d69b49b', 'room': 'Room Admin', 'date': '01-04-2024 12:03:00 ', 'temp': '42', 'location': 'Out', 'timestamp': '1711990980'})
('1711991040-0', {'id': '__export__.temp_log_71376_24a1629c', 'room': 'Room Admin', 'date': '01-04-2024 12:04:00 ', 'temp': '42', 'location': 'Out', 'timestamp': '1711991040'})
('1711991100-0', {'id': '__export__.temp_log_51693_1c235ce5', 'room': 'Room Admin', 'date': '01-04-2024 12:05:00 ',

Check the MONITOR notebook; the XRANGE command should be in the output now.

## Caching HASH data

Let's see how hash keys work with client-side caching and TTL.

First, let's create a simple hash key:

In [27]:
hash_fields = {"first_name": "John", "last_name": "Doe"}
r.hset("myhash", mapping=hash_fields)

2

time: 24.6 ms (started: 2026-01-27 13:49:34 +00:00)


Check Insight and the new key should be there. The MONITOR notebook will also show the SET command being executed.

Next, get the same key twice; only 1 output should show up in the MONITOR notebook:

In [28]:
r.hgetall("myhash")

{'first_name': 'John', 'last_name': 'Doe'}

time: 24.9 ms (started: 2026-01-27 13:51:38 +00:00)


In [29]:
r.hgetall("myhash")

{'first_name': 'John', 'last_name': 'Doe'}

time: 2.59 ms (started: 2026-01-27 13:51:43 +00:00)


Note the execution time difference between them.

Now, how does Client-Side Caching behave with TTL attributes? Let's test this by adding a new attribute to a hash, with the email attribute, set to expire in 60 seconds:

In [32]:
r.hsetex("myhash", ex=60, mapping={"email": "john.doe@gmail.com"})

1

time: 22.4 ms (started: 2026-01-27 13:55:56 +00:00)


Quickly run another GET to retrieve the updated hash; the email attribute should be there now:

In [33]:
r.hgetall("myhash")

{'email': 'john.doe@gmail.com', 'first_name': 'John', 'last_name': 'Doe'}

time: 29.4 ms (started: 2026-01-27 13:56:10 +00:00)


Wait 60 seconds for the email attribute to expire (you can use Insight to check) and run another GET:

In [34]:
r.hgetall("myhash")

{'first_name': 'John', 'last_name': 'Doe'}

time: 22.4 ms (started: 2026-01-27 13:57:25 +00:00)


This new GET command should show up in the MONITOR notebook.

## Caching JSON data

First, we need to import the dataset into Redis:

In [38]:
import os
import json
import numpy as np

from redis.commands.search.field import TagField
from redis.commands.search.field import TextField
from redis.commands.search.field import NumericField
from redis.commands.search.query import NumericFilter, Query
from redis.commands.search.index_definition import IndexDefinition, IndexType

time: 16.2 ms (started: 2026-01-27 14:11:49 +00:00)


In [36]:
prod_df = pd.read_pickle('./files/prodjson.pkl')
print(len(prod_df))
prod_df.head()

1048


,id,price,discountedPrice,articleNumber,productDisplayName,productDescription,variantName,catalogAddDate,brandName,ageGroup,...,fashionType,season,year,rating,displayCategories,masterCategory,subCategory,articleType,discount_pct,inventoryCount
0,10184,99.9,99.9,1ISB2882,Indigo Nation Men White Stripe Shirt,Composition White full sleeves shirt with dark...,Price catch,1431586973,Indigo Nation,Adults-Men,...,Fashion,Fall,2011,7,Casual Wear,Apparel,Topwear,Shirts,0.0,50
1,14519,51.9,51.9,MTSH009718,Mark Taylor Men White & Teal Blue Striped Shirt,Composition White shirt with teal blue stripes...,Stripes,1432896522,Mark Taylor,Adults-Men,...,Fashion,Fall,2011,10,Formal Wear,Apparel,Topwear,Shirts,0.0,50
2,33064,179.9,179.9,USSH2168,U.S. Polo Assn. Denim Co. Men Check Blue Shirt,Style NoteThis shirt from the US Polo Collecti...,Checks,1439815296,U.S. Polo Assn. Denim Co.,Adults-Men,...,Fashion,Summer,2012,1,Casual Wear,Apparel,Topwear,Shirts,0.0,70
3,15144,189.9,189.9,ASGS4221,Arrow Sport Men Self-Stripes White Shirts,Composition White striped shirt made of 100% c...,Stripes,1446705981,Arrow Sport,Adults-Men,...,Fashion,Fall,2011,6,Casual Wear,Apparel,Topwear,Shirts,0.0,17
4,8188,90.9,36.3,LMSH006885,Locomotive Men Solid Poplin Laidley Purple Shirts,CompositionPurple full sleeve shirt made of 10...,Solid Poplin Laidley,1410169843,LOCOMOTIVE,Adults-Men,...,Fashion,Summer,2011,7,"Shirts,Casual Wear and Clearance,Sale and Clea...",Apparel,Topwear,Shirts,60.0,56


time: 27.4 ms (started: 2026-01-27 14:07:24 +00:00)


In [37]:
for index, product in prod_df.iterrows():
  keyname = f"jsonprod:{product['id']}"
  pipe.json().set(keyname, "$", product.to_dict())
results = pipe.execute()
len(results)

1048

time: 301 ms (started: 2026-01-27 14:07:43 +00:00)


&nbsp;

<img src="https://github.com/denisabrantesredis/denisd-redis-learning-sessions/blob/main/Streams/_assets/images/callout_insight.png?raw=true" alt="Callout - Check Redis Insight"/>

Use Insight to make sure the new JSON keys are there. The MONITOR notebook will also show all 1048 keys inserted.

&nbsp;
&nbsp;

Next, let's create a search index for the product keys:

In [39]:
schema = (
    NumericField("$.id", as_name="id", sortable=True),
    NumericField("$.price", as_name="price"),
    NumericField("$.discountedPrice", as_name="discountedPrice"),
    TextField("$.articleNumber", as_name="articleNumber"),
    TextField("$.productDisplayName", as_name="productDisplayName"),
    TextField("$.productDescription", as_name="productDescription", index_missing=True, index_empty=True),
    TextField("$.variantName", as_name="variantName"),
    NumericField("$.catalogAddDate", as_name="catalogAddDate"),
    TagField("$.brandName", as_name="brandName"),
    TagField("$.ageGroup", as_name="ageGroup"),
    TagField("$.gender", as_name="gender"),
    TagField("$.baseColour", as_name="baseColour"),
    TagField("$.fashionType", as_name="fashionType"),
    TagField("$.season", as_name="season"),
    TagField("$.year", as_name="year"),
    NumericField("$.rating", as_name="rating"),
    TagField("$.displayCategories", as_name="displayCategories"),
    TagField("$.masterCategory", as_name="masterCategory"),
    TagField("$.subCategory", as_name="subCategory"),
    TextField("$.articleType", as_name="articleType"),
    NumericField("$.discount_pct", as_name="discount_pct"),
    NumericField("$.inventoryCount", as_name="inventoryCount", index_missing=True)
)
try:
    r.ft("idx:jsonprod").dropindex()
except:
    print("--> JSONProd index doesn't exist; creating it")
try:
  definition = IndexDefinition(prefix=["jsonprod:"], index_type=IndexType.JSON)
  result = r.ft("idx:jsonprod").create_index(fields=schema, definition=definition)
except Exception as ex:
    result = f"FAILED to create index: {ex}"

--> JSONProd index doesn't exist; creating it
time: 44.1 ms (started: 2026-01-27 14:11:54 +00:00)


Make sure the index is populated:

In [40]:
info = r.ft('idx:jsonprod').info()
print(f" Percent Indexed: {int(info['percent_indexed'])*100}")
print(f" Total Documents: {info['num_docs']}")

 Percent Indexed: 100
 Total Documents: 1048
time: 22.5 ms (started: 2026-01-27 14:12:22 +00:00)


We know that Client-Side Caching won't work with search commands (but we're going to test this anyway).

Let's run the standard GET command for JSON keys:

In [43]:
r.json().get("jsonprod:10110", "$")

[{'id': 10110,
  'price': 499.9,
  'discountedPrice': 499.9,
  'articleNumber': 'V58339',
  'productDisplayName': 'Reebok Women Instant White Sports Shoes',
  'productDescription': 'The reebok Instant is a versatile running shoe that features ample ventilation and offers lightweight comfort, keeping your feet fresh and comfortable all day. Added to this is the plush DMX Ride cushioning that makes it an all the more comfortable, smooth and flexible ride. Wear them when you want to run or jog or use them as casual wear. One pair of shoes, and so many uses!UpperBreathable mesh upper for ample ventilation and reduced irritationSynthetic overlays for added shoe structure, durability and styleLace-up system with synthetic laces for a snug, customized FitLushly padded tongue and well-cushioned collar for in-step comfortMesh tongue for maximum ventilationBrand logo on both sidesBrand name on tongue and behind collarMidsoleEVA midsole for lightweight responsive cushioning and shock absorptionDM

time: 5.28 ms (started: 2026-01-27 14:15:46 +00:00)


That's a lot of output. Let's say I only need the product description:

In [44]:
r.json().get("jsonprod:10110", "$.productDisplayName")

['Reebok Women Instant White Sports Shoes']

time: 4.65 ms (started: 2026-01-27 14:15:52 +00:00)


Note that both commands show up in the MONITOR notebook; a partial retrieval will not use the cache, even if the entire key is there. However, running both commands again:

In [45]:
r.json().get("jsonprod:10110", "$")

[{'id': 10110,
  'price': 499.9,
  'discountedPrice': 499.9,
  'articleNumber': 'V58339',
  'productDisplayName': 'Reebok Women Instant White Sports Shoes',
  'productDescription': 'The reebok Instant is a versatile running shoe that features ample ventilation and offers lightweight comfort, keeping your feet fresh and comfortable all day. Added to this is the plush DMX Ride cushioning that makes it an all the more comfortable, smooth and flexible ride. Wear them when you want to run or jog or use them as casual wear. One pair of shoes, and so many uses!UpperBreathable mesh upper for ample ventilation and reduced irritationSynthetic overlays for added shoe structure, durability and styleLace-up system with synthetic laces for a snug, customized FitLushly padded tongue and well-cushioned collar for in-step comfortMesh tongue for maximum ventilationBrand logo on both sidesBrand name on tongue and behind collarMidsoleEVA midsole for lightweight responsive cushioning and shock absorptionDM

time: 4.94 ms (started: 2026-01-27 14:16:38 +00:00)


In [46]:
r.json().get("jsonprod:10110", "$.productDisplayName")

['Reebok Women Instant White Sports Shoes']

time: 3.99 ms (started: 2026-01-27 14:16:40 +00:00)


We can see that none of them show up in the MONITOR output; both commands used the local cache.

Next, let's add a discount to this product (currently set at 0.0):

In [47]:
r.json().set("jsonprod:10110", "$.discount_pct", 0.15)

True

time: 23.1 ms (started: 2026-01-27 14:18:46 +00:00)


Check Insight to see the new value in the key.

Re-run both commands, and they should show up in the MONITOR notebook now, because the local cache entry was invalidated.

In [48]:
r.json().get("jsonprod:10110", "$")

[{'id': 10110,
  'price': 499.9,
  'discountedPrice': 499.9,
  'articleNumber': 'V58339',
  'productDisplayName': 'Reebok Women Instant White Sports Shoes',
  'productDescription': 'The reebok Instant is a versatile running shoe that features ample ventilation and offers lightweight comfort, keeping your feet fresh and comfortable all day. Added to this is the plush DMX Ride cushioning that makes it an all the more comfortable, smooth and flexible ride. Wear them when you want to run or jog or use them as casual wear. One pair of shoes, and so many uses!UpperBreathable mesh upper for ample ventilation and reduced irritationSynthetic overlays for added shoe structure, durability and styleLace-up system with synthetic laces for a snug, customized FitLushly padded tongue and well-cushioned collar for in-step comfortMesh tongue for maximum ventilationBrand logo on both sidesBrand name on tongue and behind collarMidsoleEVA midsole for lightweight responsive cushioning and shock absorptionDM

time: 23.2 ms (started: 2026-01-27 14:20:05 +00:00)


In [49]:
r.json().get("jsonprod:10110", "$.productDisplayName")

['Reebok Women Instant White Sports Shoes']

time: 21.9 ms (started: 2026-01-27 14:20:08 +00:00)


Check the MONITOR notebook; both commands should appear there.

Finally, let's test the search command:

In [65]:
query = Query('@productDisplayName: (Reebok Women Instant)'
                ).sort_by('id', asc=True
                ).return_fields('productDisplayName', 'price', 'id'
                ).paging(0,10)

results = r.ft("idx:jsonprod").search(query)
for result in results["results"]:
  print(result["extra_attributes"])

{'id': '4175', 'productDisplayName': 'Reebok Women Instant Black Shoe', 'price': '559.9'}
{'id': '10110', 'productDisplayName': 'Reebok Women Instant White Sports Shoes', 'price': '499.9'}
time: 24.6 ms (started: 2026-01-27 14:29:45 +00:00)


Check the MONITOR notebook for the query.

Now, repeat the search:

In [67]:
query = Query('@productDisplayName: (Reebok Women Instant)'
                ).sort_by('id', asc=True
                ).return_fields('productDisplayName', 'price', 'id'
                ).paging(0,10)

results = r.ft("idx:jsonprod").search(query)
for result in results["results"]:
  print(result["extra_attributes"])

{'id': '4175', 'productDisplayName': 'Reebok Women Instant Black Shoe', 'price': '559.9'}
{'id': '10110', 'productDisplayName': 'Reebok Women Instant White Sports Shoes', 'price': '499.9'}
time: 21.4 ms (started: 2026-01-27 14:30:30 +00:00)


As we can see, search commands are not cached by the client.

&nbsp;
&nbsp;

## Client Cache Max Size

In this lab, we set the max size of the cache to **10**. This means only 10 keys (or, to be more accurate, 10 read command outputs) will be stored at a time. We can test this with a loop.

First, let's make a list with 10 key names:

In [74]:
prod_keys = ["jsonprod:10110", "jsonprod:10111", "jsonprod:10112", "jsonprod:10164", "jsonprod:10184", "jsonprod:10193",
             "jsonprod:10214", "jsonprod:10215", "jsonprod:10220", "jsonprod:10229"]

time: 745 µs (started: 2026-01-27 14:39:46 +00:00)


Then, let's run a loop where we GET each key 3 times:

In [78]:
for i in range(3):
  for key in prod_keys:
    prod_data = r.json().get(key, "$")[0]
    print(f"--> Loop {i+1} ID: {prod_data['id']} - Product: {prod_data['productDisplayName']}")

--> Loop 1 ID: 10110 - Product: Reebok Women Instant White Sports Shoes
--> Loop 1 ID: 10111 - Product: Reebok Men Premier road supreme 2 Silver Sports Shoes
--> Loop 1 ID: 10112 - Product: Reebok Women Premier Zigfly White Sports Shoes
--> Loop 1 ID: 10164 - Product: Carrera Men Green strap witrh green dial Green Watches
--> Loop 1 ID: 10184 - Product: Indigo Nation Men  White Stripe Shirt
--> Loop 1 ID: 10193 - Product: Carrera Men Dial Green Numbers Black Watch
--> Loop 1 ID: 10214 - Product: Murcia Women Rubena Brown Handbags
--> Loop 1 ID: 10215 - Product: Murcia Women Hths Purple Handbags
--> Loop 1 ID: 10220 - Product: Murcia Women Mary Pink Handbags
--> Loop 1 ID: 10229 - Product: Murcia Women Hths Black Handbags
--> Loop 2 ID: 10110 - Product: Reebok Women Instant White Sports Shoes
--> Loop 2 ID: 10111 - Product: Reebok Men Premier road supreme 2 Silver Sports Shoes
--> Loop 2 ID: 10112 - Product: Reebok Women Premier Zigfly White Sports Shoes
--> Loop 2 ID: 10164 - Product: 

Check the MONITOR notebook; the keys should have been called only once.

Next, let's extend our list of keys to 12:

In [79]:
prod_keys = ["jsonprod:10110", "jsonprod:10111", "jsonprod:10112", "jsonprod:10164", "jsonprod:10184", "jsonprod:10193",
             "jsonprod:10214", "jsonprod:10215", "jsonprod:10220", "jsonprod:10229", "jsonprod:10230", "jsonprod:10246"]

time: 488 µs (started: 2026-01-27 14:41:16 +00:00)


Run the loop again:

In [80]:
for i in range(2):
  for key in prod_keys:
    prod_data = r.json().get(key, "$")[0]
    print(f"--> Loop {i+1} ID: {prod_data['id']} - Product: {prod_data['productDisplayName']}")

--> Loop 1 ID: 10110 - Product: Reebok Women Instant White Sports Shoes
--> Loop 1 ID: 10111 - Product: Reebok Men Premier road supreme 2 Silver Sports Shoes
--> Loop 1 ID: 10112 - Product: Reebok Women Premier Zigfly White Sports Shoes
--> Loop 1 ID: 10164 - Product: Carrera Men Green strap witrh green dial Green Watches
--> Loop 1 ID: 10184 - Product: Indigo Nation Men  White Stripe Shirt
--> Loop 1 ID: 10193 - Product: Carrera Men Dial Green Numbers Black Watch
--> Loop 1 ID: 10214 - Product: Murcia Women Rubena Brown Handbags
--> Loop 1 ID: 10215 - Product: Murcia Women Hths Purple Handbags
--> Loop 1 ID: 10220 - Product: Murcia Women Mary Pink Handbags
--> Loop 1 ID: 10229 - Product: Murcia Women Hths Black Handbags
--> Loop 1 ID: 10230 - Product: Murcia Women Claudia Brown Handbags
--> Loop 1 ID: 10246 - Product: Murcia Women Zoe Brown Handbags
--> Loop 2 ID: 10110 - Product: Reebok Women Instant White Sports Shoes
--> Loop 2 ID: 10111 - Product: Reebok Men Premier road supreme 2

Check the MONITOR notebook, see how the keys are eventually evicted from the local cache and need to be retrieved again.

&nbsp;


&nbsp;



# Congrats, this is the end of the lab!!